In [26]:
from natasha import NewsNERTagger
from natasha import MorphVocab, NewsEmbedding, NewsMorphTagger
from natasha import Doc, Segmenter
import json
import pandas as pd
import re

In [27]:
import pymorphy2
analyzer = pymorphy2.MorphAnalyzer()

In [28]:
embedding = NewsEmbedding()
segmenter = Segmenter()

In [29]:
morph_tagger = NewsMorphTagger(embedding)
morph_vocab = MorphVocab()
ner_tagger = NewsNERTagger(embedding)

In [30]:
with open("../data/corpus_as_dict.json") as f:
    docs = json.load(f)

Создадим датасет с документами

In [31]:
titles = list(docs.keys())
texts = list(docs.values())
df = pd.DataFrame.from_dict({'title':titles, 'text':texts})

In [32]:
df.sample(5)

,title,text
288,Сыр,"Сыр — пищевой продукт, получаемый из сыроприго..."
166,Мини-гольф,"Мини-гольф — миниатюрная версия гольфа, спорти..."
205,Париж,"Париж — столица и крупнейший город Франции, а ..."
319,Фильм-катастрофа,"Фильм-катастрофа или фильм катастроф — фильм, ..."
182,Нактоуз,"Нактоуз — ящик, в котором расположен судовой к..."


In [33]:
def get_ner(transcript):
  script = Doc(re.sub(r'\((.*?)\)', "", transcript))
  script.segment(segmenter)
  script.tag_morph(morph_tagger)
  for token in script.tokens:
    token.lemmatize(morph_vocab)
  script.tag_ner(ner_tagger)
  for span in script.spans:
    span.normalize(morph_vocab)
  named_ents = [(i.text, i.type, i.normal) for i in script.spans]
  normed_ents = []
  for word, tag, norm in named_ents:
    if len(word.split()) == 1 and tag == "LOC":
      for gram in range(len(analyzer.parse(word))):
        if "Geox" in analyzer.parse(word)[gram].tag:
          normed_ents.append((analyzer.parse(word)[gram].normal_form))
          break
        elif gram == len(analyzer.parse(word)) - 1:
          normed_ents.append((norm.lower().strip(".,!?;-")))
    else:
      normed_ents.append((norm.lower().strip(".,!?;-")))
  return sorted(normed_ents)

Следующая функция работает таким образом:

- текст сегментируется на токены
- на токены накладывается частеречная разметка
- токены лемматизируются
- выделяются именованные сущности
- токены нормализуются
- обрабатывается нормализация географических объектов, т.к. в процессе выяснилось, что natasha хуже их нормализует, чем pymorphy2.

In [34]:
df["named_entities"] = df.apply(lambda row: get_ner(row["text"]), axis=1)

In [35]:
df.sample(3)

,title,text,named_entities
246,Российская империя,"Российская империя — государство, существовавш...","[анна иоанновна, балтийское море, верховный та..."
186,Национальная библиотека Франции,Национальная библиотека Франции — библиотека в...,"[европа, национальная библиотека, париж, париж..."
20,Белфаст,"Белфаст — город в Великобритании, столица Севе...","[белфаст, белфаст, белфаст, великобритания, ду..."


In [36]:
has_ner = [i for i in df.index.values if df.named_entities[i]]

In [37]:
len(has_ner), df.shape[0]

(223, 363)

В 223 документах из 363 есть Именнованные сущности

In [38]:
df_ner = df[df.index.isin(has_ner)]

Будем работать именно с теми документами, где они есть

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.cluster import KMeans

In [40]:
ner_voc = []
for row in df_ner.named_entities.tolist():
  ner_voc.extend(row)
len(ner_voc), len(set(ner_voc))

(1758, 955)

Всего в корпусе 1758 сущностей, уникальных 955

In [41]:
vocabulary = sorted(set(ner_voc))
corpus = df_ner.named_entities.apply(str).tolist()

In [55]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                 ('tfid', TfidfTransformer())]).fit(corpus)
X = pipe.fit_transform(corpus)
km = KMeans(n_clusters=18, init='k-means++', max_iter=600, 
            algorithm="full", precompute_distances=True)

In [56]:
km.fit(X)

/home/alexanderlakiza/.virtualenvs/cs224/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:786: FutureWarning: 'precompute_distances' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25). It has no effect
  warnings.warn("'precompute_distances' was deprecated in version "


KMeans(algorithm='full', max_iter=600, n_clusters=18, precompute_distances=True)

Есть два вида метрик оценки качества кластеризации:

внешние - используют дополнительные знания о кластерах (распределение по кластерам, количество кластеров)
внутренние - оценивают качество полученной структуры, не используя внешнюю информацию
Т.к. у нас нет никаких дополнительных знаний (но для меньшего объема данных их можно получить экспертно при необходимости), посмотрим на внутренние метрики. В sklearn есть следующие:

In [57]:
print(metrics.silhouette_score(X, km.labels_, sample_size=1000))
print(metrics.davies_bouldin_score(X.toarray(), km.labels_))

0.0868968053447224
1.3695426552227496


силуэт: показывает, насколько объект похож на свой кластер относительно других кластеров; если значение стремится к 1 - хорошее разбиение, если к -1 - плохое, если в районе 0 - кластеры пересекаются
индекс Дэвиcа-Болдуина: оценивает расстояние от объекта кластера до центроида и расстояние между центроидами; чем ниже, тем лучше разбиение

Оценка первого параметра при разном кол-ве кластеров колеблется от 0.7 до 1, что не очень хорошо  
А вот второй параметр уменьшается по мере увеличения количества кластеров, и имеет довольно неплохой результат

In [58]:
df_ner["label"] = km.predict(X)

<ipython-input-58-20dc8af91091>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ner["label"] = km.predict(X)


In [59]:
df_ner["label"].value_counts()

2     170
13     10
14      8
3       7
12      4
15      4
17      3
6       3
8       3
5       2
11      2
16      1
0       1
10      1
1       1
7       1
4       1
9       1
Name: label, dtype: int64

In [64]:
df_ner.query("label == 3")

,title,text,named_entities,label
27,Бифштекс,"Бифштекс, стейк-филе — блюдо из жареной говяди...","[бифштекс, британия, сша, сша, сша]",3
80,Доллар США,Доллар Соединенных Штатов Америки — денежная е...,"[bloomberg, swift, соединенные штаты америка, ...",3
105,Камин,"Камин — разновидность печей-теплогенераторов, ...","[агентство по охрана окружающая среда, вашингт...",3
149,Луксор Лас-Вегас,"«Луксор Лас-Вегас» — гостиница и казино, распо...","[кларк, лас-вегас-стрип, луксор лас-вегас, нев...",3
245,Роберт Баллард,Роберт Дуэйн Баллард — американский исследоват...,"[nautilus, uss yorktown, вмс, джон кеннеди, ро...",3
254,США,"Соединенные Штаты Америки, сокращенно США, или...","[атлантический океан, вашингтон, всемирный бан...",3
343,Штаты США,Штат США — одна из 50 составляющих администрат...,"[соединенные штаты, сша, сша, сша, штат]",3


In [65]:
df_ner.query("label == 14")

,title,text,named_entities,label
15,Атм,Атмосфера — внесистемная единица измерения дав...,"[земля, международная организация законодатель...",14
51,Водород,Водород — химический элемент периодической сис...,[земля],14
93,Земля,Земля — третья по удаленности от Солнца планет...,"[арктика, венера, земля, земля, земля, земля, ...",14
124,Компас,Компас; на профессиональном жаргоне моряков — ...,[земля],14
150,Луна,Луна — единственный естественный спутник Земли...,"[земля, земля, земля, земля, земля, луна, луна...",14
152,Магнитная девиация,Магнитная девиация или магнитное отклонение — ...,[земля],14
316,Углекислый газ,Диоксид углерода или двуокись углерода — бесцв...,"[земля, земля, земля, земля, земля, земля, сол...",14
331,Человек,"Человек — общественное существо, обладающее ра...","[в. г. борзенков, земля, и. т. фролов]",14


In [66]:
df_ner.query("label == 12")

,title,text,named_entities,label
148,Лошадиная сила,Лошадиная сила — внесистемная единица мощности...,"[англия, международная система, россия, россия...",12
168,Моллюск,"Моллюски, или мягкотелые, — тип первичноротых ...",[россия],12
252,Рыбы,Рыбы — парафилетическая группа водных позвоноч...,[россия],12
256,Санкт-Петербург,Санкт-Петербург — второй по численности населе...,"[военно-морской флот, вооруженные силы, гераль...",12


In [63]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = pipe[0].get_feature_names()
for i in range(18):
  print("Cluster %d:" % i, end='')
  for ind in order_centroids[i, :10]:
    print(' %s' % terms[ind], end='')
  print()

Cluster 0: солонина японское море ехидну заморская территория залив белфаст зал слава закавказье жюль мишле жюль ардуэн-мансар жоау фернандеша лэврадур
Cluster 1: итчен тест портсмут англия великобритания европа европейский союз евразия заморская территория залив белфаст
Cluster 2: канада уайт сша европа титаник великобритания белфаст франция морзе кунард
Cluster 3: сша америка nautilus вмс штат бифштекс парадайз кларк стрип невад
Cluster 4: англия шотландия нидерланды оранский японское море ж. картье заморская территория залив белфаст зал слава закавказье
Cluster 5: атлантика северная азовское евразия шпицберген черное америка атлантический жоау фернандеша лэврадур жан кальвин
Cluster 6: гренландия америка северная земля атлантический южная австралия европа исландия антарктида
Cluster 7: непотопляемость трюм японское море ж. б. ламарком запад заморская территория залив белфаст зал слава закавказье жюль мишле
Cluster 8: испания мадрид европа африка португалия андорра марокко канарские 

### Вывод
В общем и целом, разбиение на кластеры выглядит успешным, так как примерно прослеживается тематика почти каждого кластера